<a href="https://colab.research.google.com/github/Bora-Ulu/Inflation-Technique/blob/main/Inflation_Technique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from numpy import *
from itertools import *
import time
from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix

def RefiningRows(m): #Takes a matrix as input 
                     #Returns the same matrix with the identical rows removed
    
    M=list(m)
    
    M = ascontiguousarray(M)
    uM = unique(M.view([('', M.dtype)]*M.shape[1]))
    return uM.view(M.dtype).reshape((uM.shape[0], M.shape[1]))

trial_m=array([[1,0,0,1,0],
                 [1,0,1,0,1],
                 [1,0,0,1,0]])
print(trial_m)
print('Refining:')
start = time.time()
print(RefiningRows(trial_m))
print('It takes', time.time()-start, 'seconds to calculate.')

[[1 0 0 1 0]
 [1 0 1 0 1]
 [1 0 0 1 0]]
Refining:
[[1 0 0 1 0]
 [1 0 1 0 1]]
It takes 0.0003540515899658203 seconds to calculate.


In [18]:
def IntToList(inte, base, leng):
    
    List=[int(i) for i in base_repr(inte, base)]
    
    if len(List)<leng:
        
        l=[0 for i in range(leng-len(List))]
        l.extend(List)
        List=l
        
    return List
print('The integer 1456 in the form of a list:')
start = time.time()
print(IntToList(1456, 4, 12))
print('It takes', time.time()-start, 'seconds to calculate.')

The integer 1456 in the form of a list:
[0, 0, 0, 0, 0, 0, 1, 1, 2, 3, 0, 0]
It takes 9.489059448242188e-05 seconds to calculate.


In [19]:
def ListToInt(List, Base):
    
    L=list(List)
    r = int("".join([str(i) for i in L]))
    inte = int(str(r), base = Base)
    
    return inte
a=IntToList(1456, 4, 12)
print('We have a list:')
print(a)
print('This list corresponds to the integer:')
start = time.time()
print(ListToInt(a, 4))
print('It takes', time.time()-start, 'seconds to calculate.')

We have a list:
[0, 0, 0, 0, 0, 0, 1, 1, 2, 3, 0, 0]
This list corresponds to the integer:
1456
It takes 7.510185241699219e-05 seconds to calculate.


In [20]:
def Symmetry_Op(operation, inte):#Takes the exchange opration of copy latent variables and the values of the observable variables as input 
                                   #Returns the corresponding observable values for the resultant identical probability 
    
    # The input list is of the form: [A1,A2,A3,A4,B1,B2,B3,B4,C1,C2,C3,C4]
    
    
    
    array=IntToList(inte, 4 ,12)
    
    if operation == 'X1_X2': #Exchanging \Lambda_{1}^{AB} and \Lambda_{2}^{AB}
        
        array[0],array[1]=array[1],array[0] #A1 -- A2
        array[2],array[3]=array[3],array[2] #A3 -- A4
        array[4],array[6]=array[6],array[4] #B1 -- B3
        array[6],array[7]=array[7],array[6] #B3 -- B4
        
        a=ListToInt(array , 4)
    
        return a
        
    elif operation == 'Y1_Y2': #Exchanging \Lambda_{1}^{AC} and \Lambda_{2}^{AC}
        
        array[0],array[2]=array[2],array[0] #A1 -- A3
        array[1],array[3]=array[3],array[1] #A2 -- A4
        array[8],array[9]=array[9],array[8] #C1 -- C2
        array[10],array[11]=array[11],array[10] #C3 -- C4
        
        a=ListToInt(array , 4)
        
        return a
        
    elif operation == 'Z1_Z2': #Exchanging \Lambda_{1}^{BC} and \Lambda_{2}^{BC}

        array[4],array[5]=array[5],array[4] #B1 -- B2
        array[6],array[7]=array[7],array[6] #B3 -- B4
        array[8],array[10]=array[10],array[8] #C1 -- C3
        array[9],array[11]=array[11],array[9] #C2 -- C1
        
        a=ListToInt(array , 4)
        
        return a
    
    else:
        print('Error: Enter a valid operation.')

Op='X1_X2'
inte=634567
print('Integer before the exchange:')
print(inte)
print('List before the exchange:')
print(IntToList(inte, 4, 12))
print('Integer after the exchange:')
start = time.time()
print(Symmetry_Op(Op, inte))
print('It takes', time.time()-start, 'seconds to calculate.')
print('List after the exchange:')
print(IntToList(Symmetry_Op(Op, inte) , 4, 12))

Integer before the exchange:
634567
List before the exchange:
[0, 0, 2, 1, 2, 2, 3, 2, 3, 0, 1, 3]
Integer after the exchange:
453319
It takes 9.298324584960938e-05 seconds to calculate.
List after the exchange:
[0, 0, 1, 2, 3, 2, 2, 2, 3, 0, 1, 3]


In [21]:
def Symmetric_Arrays(Op_list, inte):#Takes a list of laten variable copy index exchange operations and the values of the observable variables as input as input 
                                     #Returns all configurations of observable variable values that have identical probabilities in the form of a list
    A=[inte]
    
    permutation1=list(permutations(Op_list, 1))# A single copy index exchange operation on latent variables (X1_X2, Y1_Y2 and Z1_Z2)
    permutation2=list(permutations(Op_list, 2))# Two copy index exchange operations on latent variables
    permutation3=list(permutations(Op_list, 3))# Three copy index exchange operations on latent variables
    
    for i in range(len(permutation1)):#Lists the configurations of observable variable values that have identical probabilities for a single exchange
        
        A.append(Symmetry_Op(permutation1[i][0],inte))
    
    for i in range(len(permutation2)):#Lists the configurations of observable variable values that have identical probabilities for a two exchange operations
        
        inte1=Symmetry_Op(permutation2[i][0],inte)
        A.append(Symmetry_Op(permutation2[i][1],inte1))
        
    for i in range(len(permutation3)):#Lists the configurations of observable variable values that have identical probabilities for a three exchange operations

        inte1=Symmetry_Op(permutation3[i][0],inte)
        inte2=Symmetry_Op(permutation3[i][1],inte1)
        A.append(Symmetry_Op(permutation3[i][2],inte2))
    
    A=list(set(A))#Removes identical configurations from the list
    
    return A

Op_list=['X1_X2','Y1_Y2','Z1_Z2']
inte=634567
print('Configuration of the variable values in the form of an integer:')
print(inte)
print('All configurations that have the same probabilities as this configuration, in the form of integers:')
start = time.time()
print(Symmetric_Arrays(Op_list, inte))
print('It takes', time.time()-start, 'seconds to calculate.')

Configuration of the variable values in the form of an integer:
634567
All configurations that have the same probabilities as this configuration, in the form of integers:
[440956, 437884, 9481789, 453319, 634567, 9481171, 6339283, 6336211, 633724, 6351421]
It takes 0.00040078163146972656 seconds to calculate.


In [22]:
def RowOfaColumn(inte):
    
    c=IntToList(inte, 4 , 12)
    cc=[c[3],c[0],c[7],c[4],c[11],c[8]]
    c=[c[0],c[3],c[4],c[7],c[8],c[11]]
    c=ListToInt(c,4)
    cc=ListToInt(cc,4)
    if cc >= c:
        return c
    else:
        return cc

co=634567
print('For the column represented by the integer:')
print(co)
print('The row at which this column has its only 1 value is:')
start = time.time()
print(RowOfaColumn(co))
print('It takes', time.time()-start, 'seconds to calculate.')

For the column represented by the integer:
634567
The row at which this column has its only 1 value is:
431
It takes 0.00011396408081054688 seconds to calculate.


In [23]:
def RowList():
    
    S=[IntToList(i, 4, 6) for i in range(4**6)]
    
    return S
h=200
print('We have the configuration for the',h,'th row as:')
print('{A1,A4,B1,B4,C1,C4}')
start = time.time()
print(RowList()[h-1])
print('It takes', time.time()-start, 'seconds to calculate.')

We have the configuration for the 200 th row as:
{A1,A4,B1,B4,C1,C4}
[0, 0, 3, 0, 1, 3]
It takes 0.01241612434387207 seconds to calculate.


In [24]:
def remove_zero_rows(M):
  M = csr_matrix(M)
  num_nonzeros = diff(M.indptr)
  return M[num_nonzeros != 0]

def MDM():
    
    Op_list=['X1_X2','Y1_Y2','Z1_Z2']
    
    Checklist=[0 for i in range(4**12)]
    
    j=0
    v=0
    while j in range(len(Checklist)):
        
        if Checklist[j] == 0:
            
            S=Symmetric_Arrays(Op_list, j)
            
            if j == 0:
        
                Columns=[v for k in range(len(S))]
                v=v+1
                Rows=[RowOfaColumn(k) for k in S]
                
            else:
                
                Columns.extend((v for k in range(len(S))))
                v=v+1
                Rows.extend((RowOfaColumn(k) for k in S))
                
            for x in S:
                Checklist[x]=1
            
        j=j+1
        
    data=[1 for i in range(len(Rows))]
    m = coo_matrix((array(data), (array(Rows), array(Columns))))
    M=remove_zero_rows(m)
    M = M.toarray()
    return M

start = time.time()
M=MDM()
print('It takes', time.time()-start, 'seconds to calculate the marginal description matrix.')
print('It has',len(M),'rows and',len(M[0]),'columns')
print('Its explicit form is:')
print(M)

It takes 767.6190559864044 seconds to calculate the marginal description matrix.
It has 2080 rows and 2123776 columns
Its explicit form is:
[[1 2 2 ... 0 0 0]
 [0 2 0 ... 0 0 0]
 [0 0 2 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 2 0]
 [0 0 0 ... 1 2 1]]
